# Qiskit Tutorial

As discussed in the first lecture, the emphasis of this class is on understanding the quantum computing stack. We started at the highest level of the stack in class, briefly studying the Quantum Circuit Model of Computation and how algorithms and data are represented in the model. While this model is not alone, (there are other models such as adiabatic quantum computing, hamiltonian model of quantum computing, dissipative model for quantum computing,...etc), the quantum circuit model is the most prevalent model in industry and academic literature. As such, many software frameworks have been developed to allow software developers to interface with quantum computers using this abstraction. Including but not limited to:
- Qiskit by IBM
- Cirq and TensorFlow Quantum by Google
- Pennylane and Strawberry Fields by Xanadu
- Q# by Microsoft
- Amazon Braket by AWS
- Ocean SDK by D-Wave
- ProjectQ developed by ETH Zurich

For this tutorial and for the rest of the class we will be using Qiskit. We selected Qiskit because it is an open source framework, with a very large community and great support from IBM. Furthermore, documentation for the framework is comprehensive and easy to read, and IBM offers a lot of educational content online.

# Check your Qiskit Installation

In [ ]:
from qiskit import QuantumCircuit, transpile, qiskit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from pprint import pprint
import numpy as np
print(qiskit.__version__)

# Building Circuits in Qiskit

When implementing a quantum algorithm you usually start by building out the circuit you have designed. Let's start by building a simple circuit with one X gate.

In [ ]:
#The First step is to creat the circuit
#By default the system is in |0>
circ = QuantumCircuit(1)

After creating the circuit we can adds gates to it as follows:

In [ ]:
#Then we can gates to the circuit as follows This is the identity gate, think of as a null op
circ.id(0)

It is often helpful to vizualize your circuit as you are building it to make sure it looks as expected.

In [ ]:
#vizualize a Quantum Circuit, 'mpl' option for matplotlib vizualizaiton
circ.draw('mpl')

To measure the qubit we do the following:

In [ ]:
circ.save_statevector() # we save the state vector in order to be able to view ideal simulations
circ.measure_all()
circ.draw('mpl')

# Simulating Circuits in Qiskit: The Ideal Case

To simulate circuits you in qiskit you use hte `Aer Simulator`. We choose one of multiple methods for the simulations that determine how the simulation behaves for now we choose `method="Statevector"` to see ideal results.

In [ ]:
#Create the Simulator
simulator =  AerSimulator(method="statevector")

To start simulating a circuit we must first compile it. Understanding what it means to "compile" a quantum circuit is what is going to be covered over the course of the class, for now we just make a call to the `transpile` function.

In [ ]:
compiled_circ = transpile(circ, simulator)

Now you can run your simulation

In [ ]:
result = simulator.run(compiled_circ, shots=100).result()
pprint(result.to_dict())

We can make a histogram of the outcome of the experiment:

In [ ]:
counts = result.get_counts(0)
plot_histogram(counts)

Note that in the above example, we only run the experiment one time. How often we run the experiment does not matter because we are doing an ideal simulation and are only interested in the state vector.

### Now let's try the same set up with our X,Y,Z gates:

#### X

In [ ]:
circ_x = QuantumCircuit(1)
circ_x.x(0)
circ_x.measure_all()
circ_x.draw('mpl')

In [ ]:
compiled_circ_x = transpile(circ_x, simulator)
result = simulator.run(compiled_circ_x, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)

#### Y

In [ ]:
circ_y = QuantumCircuit(1)
circ_y.y(0)
circ_y.measure_all()
circ_y.draw('mpl')


In [ ]:

compiled_circ_y = transpile(circ_y, simulator)
result = simulator.run(compiled_circ_y, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)

#### Z

In [ ]:
circ_z = QuantumCircuit(1)
circ_z.z(0)
circ_z.measure_all()
circ_z.draw('mpl')

In [ ]:
compiled_circ_z = transpile(circ_z, simulator)
result = simulator.run(compiled_circ_z, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)

#### XX

In [ ]:
circ_xx = QuantumCircuit(1)
circ_xx.x(0)
circ_xx.x(0)
circ_xx.measure_all()
circ_xx.draw('mpl')

In [ ]:
compiled_circ_xx = transpile(circ_xx, simulator)
result = simulator.run(compiled_circ_xx, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)

#### H

In [ ]:
circ_h = QuantumCircuit(1)
circ_h.h(0)
circ_h.measure_all()
circ_h.draw('mpl')

In [ ]:
compiled_circ_h = transpile(circ_h, simulator)
result = simulator.run(compiled_circ_h, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)

### A side note on debugging quantum algorithms

When developing software, it is often helpful to run a debugger. It lets you step through the code one step at a time and view the state of your programs which consists of the values of the variables used, memory layout, .... For Qiskit we can do the following:

In [ ]:
from qiskit.quantum_info import Statevector
#We make a circuit as usual
circ_debug = QuantumCircuit(2)
#Instead of adding gates to the system using meth
gates = [
    ('h',[0]),
    ('cx',[0,1])
]
for gate in gates:
    getattr(circ_debug, gate[0])(*gate[1])
    print(Statevector.from_instruction(circ_debug))

circ_debug.draw('mpl')

Note that We are not running a simulation Here. We are simply calculating what we expect the distribution of the states to be at each time step.

# Simulating Circuits in Qiskit: Let's add Noise and mesure

Currently available quantum computers are very noise and not scalable. Understanding that noise, and ways to mitigate is a necessary part of developing a quantum application. 

In [ ]:
circ =  QuantumCircuit(2)
circ.h(0)
circ.cx(0,1)
circ.measure_all()
circ.draw('mpl')


Here is how to create a simulator for a specefic machine:

In [ ]:
from qiskit import IBMQ
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeWashingtonV2


# Get the noise model of IBMVigo
device_backend = FakeWashingtonV2()
backend_simulator = AerSimulator.from_backend(device_backend)

In [ ]:
device_backend.num_qubits

#### Let's Add Noise!

In [ ]:
N = 8
circ = QuantumCircuit(N)
circ.h(0)
for i in range(1,N):
    circ.cx(0,i)
circ.measure_all()
circ.draw('mpl')

In [ ]:
transpiled_circuit = transpile(circ, backend_simulator)
result = backend_simulator.run(transpiled_circuit, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)


We can control the simulation further by specifying our own noise models, you will do that in the Excercises Portion! We can also use seeds, but that defeats the point of quantum compuatation. It is helpful for research purposes sometimes.

# Recap

We covered:
1. How to create quantum circuits in Qiskit.
2. How to run an idea simulation, and view the evolution of the state vector at different points of execution for debugging.
3. How to simulate a machine with noise using Qiskit.

# Resources 

https://docs.quantum-computing.ibm.com

# Excercise

In [ ]:
secret_number = '11100'
bv_circ = QuantumCircuit(len(secret_number)+1,len(secret_number))

bv_circ.x(len(secret_number))
bv_circ.barrier()
bv_circ.h(range(len(secret_number)+1))
bv_circ.barrier()

#bv_circ.barrier()
for digit, query in enumerate(reversed(secret_number)):
    if query == "1":
        bv_circ.cx(digit, len(secret_number))
bv_circ.barrier()
bv_circ.h(range(len(secret_number)))
bv_circ.measure(range(len(secret_number)),range(len(secret_number)))
bv_circ.draw('mpl')

In [ ]:
transpiled_circuit = transpile(bv_circ, backend_simulator)
result = backend_simulator.run(transpiled_circuit, shots=1024).result()
counts = result.get_counts(0)
plot_histogram(counts)